In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests as rq
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

## Collecting and Scraping Data from Melon

In [111]:
titles = []
artists = []
release_dates = []
genres = []
num_comments = []
num_likes = []
num_promos = []
follower_counts = []
lyric_lengths = []

# Open chrome window to Melon Top 100 webpage to extract HTML
driver = webdriver.Chrome()
driver.get('https://www.melon.com/chart/index.htm')
time.sleep(1)  

# Find all song information links on Top 100
top_100_links = driver.find_elements(By.CSS_SELECTOR, 'a.song_info')

# Parse title, artist name, release date, genre, # likes, and star ratings of each song
for link in top_100_links:
    link.click()
    time.sleep(1)
    html = driver.page_source
    
    # Count all promo videos
    promo_links = driver.find_elements(By.CSS_SELECTOR, 'a.ellipsis.album_name')
    num_promo = len(promo_links) - 2
    # individual song parsing
    parsed = bs(html, 'html.parser')

    title = parsed.find('div', {'class:', 'song_name'}).strong.next_sibling.strip()
    artist = parsed.find('a', {'class:', 'artist_name'}).text.strip()
    release_date = parsed.find('dt', text='발매일').find_next_sibling('dd').text.strip()
    genre = parsed.find('dt', text='장르').find_next_sibling('dd').text.strip()
    num_comment = parsed.find('dt', text='댓글').find_next_sibling('dd').text.strip()[:-1]
    num_like = parsed.find('span', {'id': 'd_like_count'}).text.strip()
    # After each lyric line is a <br> tag
    lyric_length = len(parsed.find('div', {'class:', 'lyric'})('br'))
    
    artist_link = driver.find_elements(By.CSS_SELECTOR, 'a.artist_name')[0]
    artist_link.click()
    time.sleep(1)
    html = driver.page_source
    parsed = bs(html, 'html.parser')
    follower_count = parsed.find('span', {'id': 'd_like_count'}).text.strip()
    
    driver.back()
    driver.back()
    time.sleep(1)
    
    titles.append(title)
    artists.append(artist)
    release_dates.append(release_date)
    genres.append(genre)
    num_comments.append(num_comment)
    num_likes.append(num_like)
    num_promos.append(num_promo)
    follower_counts.append(follower_count)
    lyric_lengths.append(lyric_length)
    
driver.quit()

In [112]:
df = pd.DataFrame()

In [113]:
df['titles'] = titles
df['artists'] = artists
df['release_date'] = release_dates
df['genres'] = genres
df['num_comments'] = num_comments
df['num_likes'] = num_likes
df['num_promos'] = num_promos
df['follower_counts'] = follower_counts
df['lyric_lengths'] = lyric_lengths

In [114]:
display(df)

,titles,artists,release_date,genres,num_comments,num_likes,num_promos,follower_counts,lyric_lengths
0,Kitsch,IVE (아이브),2023.03.27,댄스,"1,797","70,391",2,"70,242",77
1,꽃,지수 (JISOO),2023.03.31,댄스,"1,217","48,477",1,"22,213",29
2,Ditto,NewJeans,2022.12.19,댄스,"3,014","247,035",3,"109,939",71
3,Hype boy,NewJeans,2022.08.01,댄스,"2,000","252,748",7,"109,939",81
4,OMG,NewJeans,2023.01.02,댄스,"1,695","170,524",4,"109,939",90
...,...,...,...,...,...,...,...,...,...
95,Face-off,지민,2023.03.24,랩/힙합,253,"28,869",0,"142,945",58
96,발걸음,DK(디셈버),2023.03.30,발라드,43,"4,273",2,"6,221",34
97,우연히 봄,로꼬,2015.04.08,"발라드, 랩/힙합, 국내드라마",742,"162,161",3,"116,172",86
98,Golden Hour,마크 (MARK),2023.04.07,랩/힙합,692,"23,983",2,"46,614",91


In [115]:
print(num_comments)
print(num_likes)

['1,797', '1,217', '3,014', '2,000', '1,695', '1,603', '2,123', '3,127', '633', '359', '1,121', '383', '4,221', '398', '236', '8,566', '738', '171', '2,543', '3,559', '1,956', '3,777', '289', '2,728', '55', '1,556', '2,231', '169', '7,233', '1,872', '256', '974', '5,993', '240', '190', '810', '19,340', '1,318', '557', '12,931', '92', '1,047', '453', '1,772', '765', '479', '508', '2,285', '453', '2,113', '509', '7,901', '294', '518', '77', '276', '916', '133', '966', '2,053', '884', '407', '37', '395', '3,610', '198', '131', '4,367', '402', '758', '3,099', '893', '417', '934', '1,620', '1,823', '327', '218', '484', '703', '60', '220', '5,633', '2,270', '609', '1,073', '337', '608', '93', '114', '280', '1,249', '155', '337', '61', '253', '43', '742', '692', '48']
['70,391', '48,477', '247,035', '252,748', '170,524', '72,346', '49,358', '293,700', '32,206', '146,836', '28,195', '51,187', '126,990', '31,790', '31,792', '192,140', '30,569', '20,844', '218,680', '222,753', '155,089', '90,092

In [116]:
df.to_csv('top100_melon.csv')

In [117]:
df = pd.read_csv('top100_melon.csv')
display(df)

,Unnamed: 0,titles,artists,release_date,genres,num_comments,num_likes,num_promos,follower_counts,lyric_lengths
0,0,Kitsch,IVE (아이브),2023.03.27,댄스,"1,797","70,391",2,"70,242",77
1,1,꽃,지수 (JISOO),2023.03.31,댄스,"1,217","48,477",1,"22,213",29
2,2,Ditto,NewJeans,2022.12.19,댄스,"3,014","247,035",3,"109,939",71
3,3,Hype boy,NewJeans,2022.08.01,댄스,"2,000","252,748",7,"109,939",81
4,4,OMG,NewJeans,2023.01.02,댄스,"1,695","170,524",4,"109,939",90
...,...,...,...,...,...,...,...,...,...,...
95,95,Face-off,지민,2023.03.24,랩/힙합,253,"28,869",0,"142,945",58
96,96,발걸음,DK(디셈버),2023.03.30,발라드,43,"4,273",2,"6,221",34
97,97,우연히 봄,로꼬,2015.04.08,"발라드, 랩/힙합, 국내드라마",742,"162,161",3,"116,172",86
98,98,Golden Hour,마크 (MARK),2023.04.07,랩/힙합,692,"23,983",2,"46,614",91


## A Little Cleaning and Manipulation

In [118]:
df['title_lengths'] = df.titles.apply(lambda x: len(x))

In [119]:
df = df[['titles', 'title_lengths', 'artists', 'release_date', 'genres', 'num_likes', 'num_comments', 'num_promos', 'follower_counts', 'lyric_lengths']]

In [120]:
# I'm simplifying the date calculations because the specifics aren't quite relevant to my goal.
# We'll assume all months have 30 days, except February, which has 28 days.
def quantize_date(date):
    ymd = date.split('.')
    if (ymd[1] == '02'):
        return ( 30 + int(ymd[2])) / 358
    else:
        return ( (30 * (int(ymd[1]) - 2)) + int(ymd[2]) + 28) / 358

In [ ]:
# Parsing the numerical values to integers
df['num_comments'] = df.num_comments.apply(lambda x: int(x.replace(',', '')))
df['num_likes'] = df.num_likes.apply(lambda x: int(x.replace(',', '')))
df['num_promos'] = df.num_promos.apply(lambda x: int(x))
df['follower_counts'] = df.follower_counts.apply(lambda x: int(x.replace(',', '')))
df['lyric_lengths'] = df.lyric_lengths.apply(lambda x: int(x))
df['release_date_quantized'] = df.release_date.apply(lambda x: quantize_date(x))

In [121]:
display(df)

,titles,title_lengths,artists,release_date,genres,num_likes,num_comments,num_promos,follower_counts,lyric_lengths,release_date_quantized
0,Kitsch,6,IVE (아이브),2023.03.27,댄스,70391,1797,2,70242,77,0.237430
1,꽃,1,지수 (JISOO),2023.03.31,댄스,48477,1217,1,22213,29,0.248603
2,Ditto,5,NewJeans,2022.12.19,댄스,247035,3014,3,109939,71,0.969274
3,Hype boy,8,NewJeans,2022.08.01,댄스,252748,2000,7,109939,81,0.583799
4,OMG,3,NewJeans,2023.01.02,댄스,170524,1695,4,109939,90,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
95,Face-off,8,지민,2023.03.24,랩/힙합,28869,253,0,142945,58,0.229050
96,발걸음,3,DK(디셈버),2023.03.30,발라드,4273,43,2,6221,34,0.245810
97,우연히 봄,5,로꼬,2015.04.08,"발라드, 랩/힙합, 국내드라마",162161,742,3,116172,86,0.268156
98,Golden Hour,11,마크 (MARK),2023.04.07,랩/힙합,23983,692,2,46614,91,0.265363


## Exploratory Data Analysis!

In [122]:
df.genres.value_counts()

댄스                  30
발라드                 23
랩/힙합                10
록/메탈                 9
POP                  8
R&B/Soul             5
발라드, 국내드라마           4
성인가요/트로트             3
포크/블루스               2
일렉트로니카               1
J-POP                1
재즈                   1
발라드, 인디음악            1
인디음악, 포크/블루스         1
발라드, 랩/힙합, 국내드라마     1
Name: genres, dtype: int64

In [123]:
# Comparison of average number of likes across genres
print(df.groupby('genres')['num_likes'].mean().sort_values(ascending=False))

genres
발라드, 국내드라마          262428.000000
발라드, 랩/힙합, 국내드라마    162161.000000
인디음악, 포크/블루스        140734.000000
댄스                  135944.733333
POP                 131894.000000
록/메탈                131715.333333
랩/힙합                124250.700000
R&B/Soul             98201.400000
발라드, 인디음악            76748.000000
성인가요/트로트             72772.000000
발라드                  64869.826087
J-POP                63400.000000
포크/블루스               46466.500000
재즈                   46365.000000
일렉트로니카               31792.000000
Name: num_likes, dtype: float64


As expected, it seems ballad and dance/K-pop are trending. Surprisingly, indie/blues music also seems to be quite popular.

In [129]:
# Let's see the numerical value correlations
df.corr()

,title_lengths,num_likes,num_comments,num_promos,follower_counts,lyric_lengths,release_date_quantized
title_lengths,1.000000,-0.001281,-0.060178,-0.002606,-0.008566,0.189542,0.092970
num_likes,-0.001281,1.000000,0.641789,0.285550,0.575633,0.185581,0.142133
num_comments,-0.060178,0.641789,1.000000,0.343261,0.735231,0.207917,0.072361
num_promos,-0.002606,0.285550,0.343261,1.000000,0.191957,0.233662,0.024858
follower_counts,-0.008566,0.575633,0.735231,0.191957,1.000000,0.188359,0.116409
lyric_lengths,0.189542,0.185581,0.207917,0.233662,0.188359,1.000000,0.125733
release_date_quantized,0.092970,0.142133,0.072361,0.024858,0.116409,0.125733,1.000000


Number of likes and number comments are positively correlated, nothing particularly special there.<br>Being released later in the year is slightly related to higher number of likes, but I wonder why...?<br>Longer title length seems to very very slightly correlate with higher number of likes, but it's probably not significant.

## We can explore the relationship between release date, genre, and number of likes a bit more. Koreans might prefer a certain genre during certain times of the year.

In [125]:
def to_season(date):
    ymd = date.split('.')
    if (int(ymd[1]) >= 12 or int(ymd[1]) <= 2):
        return 'winter'
    elif (int(ymd[1]) >= 3 and int(ymd[1]) <= 5):
        return 'spring'
    elif (int(ymd[1]) >= 6 and int(ymd[1]) <= 8):
        return 'summer'
    elif (int(ymd[1]) >= 9 and int(ymd[1]) <= 11):
        return 'autumn'
    else:
        return 'na'
    
df['release_season'] = df.release_date.apply(lambda x: to_season(x))

In [126]:
display(df)

,titles,title_lengths,artists,release_date,genres,num_likes,num_comments,num_promos,follower_counts,lyric_lengths,release_date_quantized,release_season
0,Kitsch,6,IVE (아이브),2023.03.27,댄스,70391,1797,2,70242,77,0.237430,spring
1,꽃,1,지수 (JISOO),2023.03.31,댄스,48477,1217,1,22213,29,0.248603,spring
2,Ditto,5,NewJeans,2022.12.19,댄스,247035,3014,3,109939,71,0.969274,winter
3,Hype boy,8,NewJeans,2022.08.01,댄스,252748,2000,7,109939,81,0.583799,summer
4,OMG,3,NewJeans,2023.01.02,댄스,170524,1695,4,109939,90,0.000000,winter
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Face-off,8,지민,2023.03.24,랩/힙합,28869,253,0,142945,58,0.229050,spring
96,발걸음,3,DK(디셈버),2023.03.30,발라드,4273,43,2,6221,34,0.245810,spring
97,우연히 봄,5,로꼬,2015.04.08,"발라드, 랩/힙합, 국내드라마",162161,742,3,116172,86,0.268156,spring
98,Golden Hour,11,마크 (MARK),2023.04.07,랩/힙합,23983,692,2,46614,91,0.265363,spring


In [127]:
print(df.groupby('release_season')['num_likes'].mean().sort_values(ascending=False))

release_season
summer    196425.545455
winter    124134.863636
autumn    120425.777778
spring     89128.591837
Name: num_likes, dtype: float64


Songs released around mid-year seem to be more popular.
Notes: Scrape artist follower count, lyric length (count number of br), song length

In [130]:
print(df.groupby('release_season')['num_comments'].mean().sort_values(ascending=False))

release_season
summer    3205.090909
winter    2026.909091
autumn    1508.333333
spring    1166.836735
Name: num_comments, dtype: float64


In [131]:
print(df.groupby('release_season')['lyric_lengths'].mean().sort_values(ascending=False))

release_season
summer    80.545455
autumn    62.055556
winter    61.636364
spring    53.591837
Name: lyric_lengths, dtype: float64


In [132]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
df = df.drop(['titles', 'artists', 'release_date', 'num_comments'], axis=1)

In [166]:
# Convert categorical to numerical
df = pd.get_dummies(df)

X = df.drop('num_likes', axis=1)
y = df['num_likes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.56, random_state=42)

# Training our model
rf = RandomForestRegressor(n_estimators=1000, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

Mean Squared Error: 9309170068.13438
